## Import

In [1]:
# basic libray
import pickle
import numpy as np
import random
random.seed(0)
import pandas as pd
import warnings; warnings.filterwarnings("ignore")

# surprise Library import
from surprise import Dataset
from surprise import Reader

## Read data

In [2]:
data_path = '../data/'

pdde = pd.read_csv(data_path + "DC_LPOINT_PDDE.csv")                # [DC] 유통사 상품 구매 내역

## Selection
연관규칙탐사(장바구니 분석)은 한 결제 내 같이 산 상품들간의 연관성을 분석한다.<br>
영수증 번호가 같은 구매내역을 한 결제로 보았으나 영수증 번호당 구매한 상품항목 종류수를 구하였을 때 한 상품항목만 구매한 데이터가 압도적으로 많다.<br>
이에 `고객`별 `제휴사, 채널, 매장, 날짜, 시간`이 같은 구매내역을 한 결제로 보고 한 결제당 상품항목종류수가 2 이상인 데이터만을 사용하도록 한다.

In [3]:
# 영수증 번호가 같은 구매내역을 동일한 결제로 볼 경우의 결제당 상품항목종류수
set_rct = pdde.groupby('rct_no')['pd_c'].nunique()
set_rct = pd.merge(set_rct.reset_index(), pdde.drop_duplicates('rct_no')[['rct_no','chnl_dv']], on='rct_no')

display(set_rct.pd_c.value_counts().head())
print('연관분석에 부합하지 않는 데이터수:', set_rct[set_rct['pd_c'] == 1].shape[0])
display(set_rct[set_rct['pd_c'] == 1].chnl_dv.value_counts())
print('연관분석에 부합하는 데이터수:', set_rct[set_rct['pd_c'] > 1].shape[0])
display(set_rct[set_rct['pd_c'] > 1].chnl_dv.value_counts())

1    736960
2    144685
3     87047
4     60980
5     46372
Name: pd_c, dtype: int64

연관분석에 부합하지 않는 데이터수: 736960


1    679751
2     57209
Name: chnl_dv, dtype: int64

연관분석에 부합하는 데이터수: 529629


1    489148
2     40481
Name: chnl_dv, dtype: int64

In [4]:
# 고객, 제휴사, 채널, 매장, 날짜, 시간이 같은 구매내역을 동일한 결제로 볼 때의 결제당 상품항목종류수
pdde.br_c.fillna('ONLINE', inplace=True)
set_info = pdde.groupby(['cust','cop_c','chnl_dv','br_c','de_dt','de_hr'])['pd_c'].nunique()

display(set_info.value_counts().head())
print('연관분석에 부합하지 않는 데이터수:', set_info[set_info == 1].shape[0])
display(set_info[set_info == 1].reset_index().chnl_dv.value_counts())
print('연관분석에 부합하는 데이터수:', set_info[set_info > 1].shape[0])
display(set_info[set_info > 1].reset_index().chnl_dv.value_counts())

1    469024
2    181353
3    100529
4     65879
5     48386
Name: pd_c, dtype: int64

연관분석에 부합하지 않는 데이터수: 469024


1    418996
2     50028
Name: chnl_dv, dtype: int64

연관분석에 부합하는 데이터수: 592412


1    551112
2     41300
Name: chnl_dv, dtype: int64

In [5]:
# 조건에 부합한 데이터만 다룬다.
ease = set_info[set_info > 1].index
pdde = pdde.set_index(['cust','cop_c','chnl_dv','br_c','de_dt','de_hr']).loc[ease].reset_index()

## User-item 행렬
[참고] https://ok-lab.tistory.com/93

In [6]:
# surprise 패키지 input을 위해 user-item 행렬 만들기 (surprise 패키지는 행렬의 unstack형식 사용)
user_item_matrix = pd.pivot_table(data = pdde, index = "pd_nm", columns = "cust",
                    values = "buy_ct", aggfunc = "sum")   # 고객의 상품별 구매횟수 -> 상품에 대한 호감도 -> 평점데이터 대체
user_item_matrix = user_item_matrix.apply(lambda x : x/x.sum())  # 고객별 구매횟수에 대한 정규화
user_item_matrix = user_item_matrix.unstack().reset_index().rename(columns = {'cust':"custID", "pd_nm":"item", 0:"buy_ct"})
user_item_matrix = user_item_matrix.dropna()
user_item_matrix.head(5)

,custID,item,buy_ct
17,M000034966,가공조미료,0.049180
25,M000034966,간장,0.016393
30,M000034966,감자스낵,0.032787
109,M000034966,과일음료,0.016393
142,M000034966,국산맥주,0.098361


## 변환
suprise 패키지의 input용으로 데이터 자료형을 변경한다.

In [7]:
# 변환 전에 main.ipynb에서 모델을 적용해볼 고객 10명 추출한다.
val_cust_list = []
for _ in range(10):
    val_cust_list.append(pdde.cust.unique()[random.randint(0, pdde.cust.nunique())])
    
user_item_matrix.query("custID in @val_cust_list").to_csv(data_path + "simulation/collab_fitering_valset.csv")

In [8]:
# surprise 패키지 input용으로 data 변환한다.
input_data = user_item_matrix.query("custID not in @val_cust_list")
reader = Reader(rating_scale=(0.0, 1.))
data = Dataset.load_from_df(input_data, reader)

## Save Data

In [9]:
pickle.dump(data, open(data_path+'user-item matrix', 'wb'))